#Load Cloud Drive

In [ ]:
from google.colab import drive
import os
os.chdir('../data/Movie_Trailer_Dataset/21K_movie_record')
!ls

tmdb_metadata_2000_1000.csv  tmdb_metadata_2011_1000.csv
tmdb_metadata_2001_1000.csv  tmdb_metadata_2012_1000.csv
tmdb_metadata_2002_1000.csv  tmdb_metadata_2013_1000.csv
tmdb_metadata_2003_1000.csv  tmdb_metadata_2014_1000.csv
tmdb_metadata_2004_1000.csv  tmdb_metadata_2015_1000.csv
tmdb_metadata_2005_1000.csv  tmdb_metadata_2016_1000.csv
tmdb_metadata_2006_1000.csv  tmdb_metadata_2017_1000.csv
tmdb_metadata_2007_1000.csv  tmdb_metadata_2018_1000.csv
tmdb_metadata_2008_1000.csv  tmdb_metadata_2019_1000.csv
tmdb_metadata_2009_1000.csv  tmdb_metadata_2020_1000.csv
tmdb_metadata_2010_1000.csv


#TMDB

In [ ]:
!pip install tmdbv3api

In [ ]:
from tmdbv3api import TMDb
from tmdbv3api import Movie,Discover,Genre,Keyword
import csv
import numpy as np
import pandas


API_V3_KEY = '55c9a195575d446d4c9271aa210bdd81'
tmdb = TMDb()
tmdb.api_key = API_V3_KEY
tmdb.language = 'en'
tmdb.debug = True
genres_list = {
    28:'Action',
    12:'Adventure',
    16:'Animation',
    35:'Comedy',
    80:'Crime',
    99:'Documentary',
    18:'Drama',
    14:'Fantasy',
    36:'History',
    27:'Horror',
    37:'Western',
    53:'Thriller',
    878:'Science Fiction',
    10751:'Family',
    10402:'Music',
    9648:'Mystery',
    10749:'Romance',
    10770:'TV Movie',
    10752:'War'
}

def get_genres_list():
  genre = Genre()
  genres = genre.movie_list()
  genres_list = {}
  for g in genres:
    genres_list[g['id']] = g['name']
  return genres_list

def get_movie_information(m):
  movie = Movie()
  info = {'id' : None,
          'title' : None,
          'release_date' : None,
          'genres': None,
          'video_key':None,
          'overview':None,
          'trailer_url' : None,
          'keywords' : None,
          'popularity' : None,
          'vote_count' : None,
          'vote_average' : None
          }
  try:
    info['id'] = m.id
    info['title'] = m.title
    info['release_date'] = m.release_date
    info['popularity'] = m.popularity
    info['vote_count'] = m.vote_count
    info['vote_average'] = m.vote_average
  except Exception as e:
    print('Cannot find movie id is :',m.id)
  try:
    genres = []
    ids = m.genre_ids
    for id in ids:
      genres.append(genres_list[id])
    info['genres'] = genres
  except Exception as e:
    print('Cannot find movie genres for :', m.id)

  try:
    movie_keywords = movie.keywords(m.id)['keywords']
    if movie_keywords is not None:
      keyws = []
      for keyw in movie_keywords:
        keyws.append(keyw['name'])
      info['keywords'] = keyws
    else:
      info['keywords'] = None

  except Exception as e:
    print('Cannot find keywords for :', m.id)
  try:
    info['overview'] = m.overview
  except Exception as e:
    info['overview'] = None

  try:
    trailer_keys = movie.videos(m.id)
    if trailer_keys is not None:
      for trailer_key in trailer_keys:
        if trailer_key['type'] == 'Trailer':
          if trailer_key['site'] == 'YouTube':
            info['trailer_url'] = 'https://www.youtube.com/watch?v='+ trailer_key['key']
            info['video_key'] = trailer_key['key']
            break
        else:
          info['trailer_url'] = None
    else:
      info['trailer_url'] = None
  except Exception as e:
    print('Can not find trailer url for :', m.id)
  return info


def discover_movies(year, num):
  meta_data =  [['id','title','release_date','genres','video_key','overview','trailer_url','keywords','popularity','vote_count','vote_average']]
  for i in range(int(num/20)):
    movie = Movie()
    discover = Discover()
    m_20 = discover.discover_movies({
        'primary_release_date.gte': '{y}-01-01'.format(y = year),
        'primary_release_date.lte': '{y}-12-31'.format(y = year),
        'page': '{page}'.format(page = i + 1),
        'sort_by':'vote_count.desc',
        }
    )
    for m in m_20:
      info = get_movie_information(m)
      meta_data.append(info.values())
  return meta_data

  
def save_meta_data(year,num):
  print('Writing file to csv')
  meta = discover_movies(year,num)
  with open('tmdb_metadata_{y}_{n}.csv'.format(y = year,n = num),'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(meta)
  print('In {y}, total number:{n}'.format(y = year,n = num))



#Main

In [ ]:
#Based on the release date and sorted by popularity, 20 movies for each page (num has to be Multiples of 20 )
def main(year_s,year_e,num):
  for i in range(year_s,year_e + 1,1):
    save_meta_data(i,num)
main(2000,2009,1000)

Writing file to csv
In 2000, total number:1000
Writing file to csv
In 2001, total number:1000
Writing file to csv
In 2002, total number:1000
Writing file to csv
In 2003, total number:1000
Writing file to csv
Cannot find keywords for : 53721
Can not find trailer url for : 53721
In 2004, total number:1000
Writing file to csv
In 2005, total number:1000
Writing file to csv
In 2006, total number:1000
Writing file to csv
In 2007, total number:1000
Writing file to csv
In 2008, total number:1000
Writing file to csv
In 2009, total number:1000
